In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# Define the GRU model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # Fully connected layer for output

    def forward(self, x):
        # GRU forward pass
        out, _ = self.gru(x)  # Ignore hidden state output
        out = out[:, -1, :]  # Take the output from the last time step
        out = self.fc(out)  # Pass through the fully connected layer
        return out
    
# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)  # Fully connected layer for output

    def forward(self, x):
        # LSTM forward pass
        out, _ = self.lstm(x)  # Ignore hidden state and cell state output
        out = out[:, -1, :]  # Take the output from the last time step
        out = self.fc(out)  # Pass through the fully connected layer
        return out

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

data = pd.read_excel('FinancialMarketData.xlsx')

features = [
    "XAU BGNL", "BDIY", "CRY", "DXY", "JPY", "GBP", "Cl1", "VIX",
    "USGG30YR", "GT10", "USGG2YR", "USGG3M", "GTDEM30Y", "GTDEM10Y",
    "GTITL30YR", "GTITL10YR", "GTJPY30YR", "GTJPY10YR", "GTGBP30Y",
    "US0001M", "MXUS", "MXEU", "MXJP", "MXBR", "MXRU", "MXIN", "MXCN"
]
data = data[features]

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

def create_sequences(data, sequence_length=30):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length])
    return np.array(X), np.array(y)

sequence_length = 30
X, y = create_sequences(data_scaled, sequence_length)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(TimeSeriesDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(TimeSeriesDataset(X_val, y_val), batch_size=32, shuffle=False)
test_loader = DataLoader(TimeSeriesDataset(X_test, y_test), batch_size=32, shuffle=False)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)  
        out = self.fc(h_n[-1]) 
        return out

# Define GRU Model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.2):
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, h_n = self.gru(x)  
        out = self.fc(h_n[-1])  
        return out


input_size = X_train.shape[2]
hidden_size = 64
output_size = y_train.shape[1]
num_layers = 2
dropout = 0.2
num_epochs = 50
learning_rate = 0.001


lstm_model = LSTMModel(input_size, hidden_size, output_size, num_layers, dropout)
gru_model = GRUModel(input_size, hidden_size, output_size, num_layers, dropout)

criterion = nn.MSELoss()
lstm_optimizer = torch.optim.Adam(lstm_model.parameters(), lr=learning_rate)
gru_optimizer = torch.optim.Adam(gru_model.parameters(), lr=learning_rate)


def train_model(model, optimizer, train_loader, val_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation loss
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")


print("Training LSTM model...")
train_model(lstm_model, lstm_optimizer, train_loader, val_loader, num_epochs)

print("Training GRU model...")
train_model(gru_model, gru_optimizer, train_loader, val_loader, num_epochs)


def evaluate_model(model, test_loader):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            test_loss += loss.item()
    return test_loss / len(test_loader)


lstm_test_loss = evaluate_model(lstm_model, test_loader)
gru_test_loss = evaluate_model(gru_model, test_loader)

print(f"LSTM Test Loss: {lstm_test_loss:.4f}")
print(f"GRU Test Loss: {gru_test_loss:.4f}")
